In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.cuda.get_device_name(0))  # Name of the first GPU


True
1
NVIDIA GeForce GTX 1650


In [2]:
from ultralytics import YOLO

In [3]:
from ultralytics import YOLO
import os

# Load a pre-trained YOLO model (Use 'yolov8l.pt' for better accuracy)
model = YOLO("yolov8s.pt")  # Change to 'yolov8x.pt' for extra-large model

# Dynamically count dataset images
train_path = "C:/Serve_IQ/train/images"
dataset_size = len(os.listdir(train_path))  # Count images in train folder

# Adaptive Training Parameters
batch_size = 16 if dataset_size > 1500 else 16  # Increase batch if dataset is large
epochs = max(50, dataset_size // batch_size)  # Minimum 50 epochs, adaptive based on data size

# Train the model
model.train(
    data="C:\Serve_IQ\data.yaml",  # Path to dataset YAML
    epochs=epochs,  # Adaptive epochs
    batch=batch_size,  # Adjusted for dataset size
    imgsz=640,  # Standard YOLO image size
    optimizer="AdamW",  # Efficient optimizer
    lr0=0.0005,  # Stable learning rate
    patience=20,  # Early stopping
    mosaic=1,  # Enable mosaic augmentation
    hsv_h=0.02, hsv_s=0.6, hsv_v=0.4,  # Color augmentation
    translate=0.2, scale=0.6, shear=0.2,  # Geometric transformations
    degrees=25,  # Rotation augmentation
    weight_decay=0.0003,  # Regularization to reduce overfitting
    device="cuda",  # Use GPU
    workers=8,  # Faster data loading
    project="brain_tumor_detection",  # Save results in this folder
    name="adaptive_training"  # Training run name
)

# Validate the model
model.val()

# Export the trained model for deployment
model.export(format="onnx")  # Save in ONNX format for deployment

<>:17: SyntaxWarning: invalid escape sequence '\S'
<>:17: SyntaxWarning: invalid escape sequence '\S'
C:\Users\abish\AppData\Local\Temp\ipykernel_9604\857645034.py:17: SyntaxWarning: invalid escape sequence '\S'
  data="C:\Serve_IQ\data.yaml",  # Path to dataset YAML
100%|██████████| 21.5M/21.5M [00:08<00:00, 2.55MB/s]
C:\Users\abish\AppData\Local\Temp\ipykernel_9604\857645034.py:17: SyntaxWarning: invalid escape sequence '\S'
  data="C:\Serve_IQ\data.yaml",  # Path to dataset YAML

KeyboardInterrupt



In [4]:
# Load YOLO Model (YOLOv8 Nano)
model = YOLO("C:\\Serve_IQ\\Tumor.pt") 

In [5]:
data_yaml = r"C:\Serve_IQ\data.yaml"

In [6]:
# ===========================
# 2. Validate on Test Dataset
# ===========================
print("\n=== Validating Model on Test Set ===")
Validate_results = model.val(data=data_yaml, split="val")  # Validate on test set
print(f"Validation Metrics: {Validate_results.box}")



=== Validating Model on Test Set ===
Ultralytics 8.3.75  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)


Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Serve_IQ\valid\labels.cache... 502 images, 0 backgrounds, 0 corrupt: 100%|██████████| 502/502 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:04<00:00,  6.77it/s]


                   all        502        505      0.894      0.882      0.927       0.57
                Glioma        161        163      0.851      0.828       0.88      0.528
            Meningioma        124        125      0.956      0.866      0.945      0.556
              No Tumor         99         99      0.879      0.909      0.935      0.692
             Pituitary        118        118      0.892      0.924      0.949      0.504
Speed: 0.6ms preprocess, 4.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\val3
Validation Metrics: ultralytics.utils.metrics.Metric object with attributes:

all_ap: array([[    0.88006,     0.86572,     0.84489,     0.79846,     0.73073,     0.53582,     0.40039,     0.17221,    0.050092,   0.0014627],
       [    0.94521,     0.93454,     0.90923,     0.85073,     0.76535,     0.65636,      0.2995,     0.16971,    0.024921,   0.0023162],
       [    0.93481,     0.93351,     0.92299,     0.91915,     0.90136,   

In [7]:
import numpy as np

In [9]:

print("\n=== Running Model Inference on Test Set ===")
pred_results = model.predict(source="C:/Serve_IQ/test/images", save=False)

# Extract class labels from predictions
predictions = []
test_labels = []

for result in pred_results:
    predicted_classes = result.boxes.cls.cpu().numpy().astype(int)  # Predicted class labels
    
    # Ground truth class labels (corrected)
    true_classes = [int(cls) for cls in result.boxes.cls.cpu().numpy()]  

    predictions.extend(predicted_classes)
    test_labels.extend(true_classes)

# Convert to NumPy arrays
predictions = np.array(predictions)
test_labels = np.array(test_labels)

# Ensure labels match predictions length
min_len = min(len(test_labels), len(predictions))
test_labels = test_labels[:min_len]
predictions = predictions[:min_len]


=== Running Model Inference on Test Set ===

image 1/246 C:\Serve_IQ\test\images\BrainTumorDSTest_1.jpg: 640x640 1 Glioma, 74.4ms
image 2/246 C:\Serve_IQ\test\images\BrainTumorDSTest_10.jpg: 640x640 1 Glioma, 74.4ms
image 3/246 C:\Serve_IQ\test\images\BrainTumorDSTest_100.jpg: 640x640 1 Meningioma, 74.2ms
image 4/246 C:\Serve_IQ\test\images\BrainTumorDSTest_101.jpg: 640x640 1 Meningioma, 1 No Tumor, 5.3ms
image 5/246 C:\Serve_IQ\test\images\BrainTumorDSTest_102.jpg: 640x640 1 Meningioma, 8.9ms
image 6/246 C:\Serve_IQ\test\images\BrainTumorDSTest_103.jpg: 640x640 1 Meningioma, 5.3ms
image 7/246 C:\Serve_IQ\test\images\BrainTumorDSTest_104.jpg: 640x640 1 Glioma, 1 Meningioma, 7.2ms
image 8/246 C:\Serve_IQ\test\images\BrainTumorDSTest_105.jpg: 640x640 1 Glioma, 6.1ms
image 9/246 C:\Serve_IQ\test\images\BrainTumorDSTest_106.jpg: 640x640 2 Meningiomas, 1 No Tumor, 11.8ms
image 10/246 C:\Serve_IQ\test\images\BrainTumorDSTest_107.jpg: 640x640 1 Meningioma, 5.5ms
image 11/246 C:\Serve_IQ\test

In [10]:
!pip install -U scikit-learn matplotlib seaborn


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\PythonServer\testing_rat\test\Scripts\python.exe -m pip install --upgrade pip


In [11]:
!pip install -U matplotlib seaborn
!pip install -U scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\PythonServer\testing_rat\test\Scripts\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\PythonServer\testing_rat\test\Scripts\python.exe -m pip install --upgrade pip


In [12]:
!pip install scipy


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\PythonServer\testing_rat\test\Scripts\python.exe -m pip install --upgrade pip


In [ ]:
!pip uninstall scikit-learn
!pip install scikit-learn


^C


In [17]:
from sklearn.metrics import confusion_matrix
from scipy.stats import chi2_contingency
import numpy as np

# ===========================
# 4. Bias Evaluation & Testing
# ===========================
protected_attribute = np.random.randint(0, 2, size=len(test_labels))  # Dummy protected attribute

# Compute Fairness Metrics
def fairness_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    if conf_matrix.size != 4:
        print("Skipping fairness metrics due to missing class predictions.")
        return
    
    tn, fp, fn, tp = conf_matrix.ravel()
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0

    print(f"False Positive Rate: {false_positive_rate:.4f}")
    print(f"False Negative Rate: {false_negative_rate:.4f}")

# Disparate Impact Analysis
def disparate_impact(y_true, y_pred, protected_attr):
    group_0 = y_pred[protected_attr == 0]
    group_1 = y_pred[protected_attr == 1]

    pr_0 = np.mean(group_0) if len(group_0) > 0 else 0
    pr_1 = np.mean(group_1) if len(group_1) > 0 else 0

    di_ratio = pr_1 / pr_0 if pr_0 > 0 else 0
    print(f"Disparate Impact Ratio: {di_ratio:.4f}")

# Chi-Square Bias Test
def chi_square_bias_test(y_true, y_pred, protected_attr):
    contingency_table = np.array([
        [sum((y_pred == 1) & (protected_attr == 0)), sum((y_pred == 0) & (protected_attr == 0))],
        [sum((y_pred == 1) & (protected_attr == 1)), sum((y_pred == 0) & (protected_attr == 1))]
    ])

    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    print(f"Chi-Square Test p-value: {p_value:.4f}")

    if p_value < 0.05:
        print("⚠️ Bias Detected!")
    else:
        print("✅ No Significant Bias Found.")

# Run Bias Analysis
print("\n=== Bias Analysis ===")
fairness_metrics(test_labels, predictions)
disparate_impact(test_labels, predictions, protected_attribute)
chi_square_bias_test(test_labels, predictions, protected_attribute)


ModuleNotFoundError: No module named 'sklearn'

In [16]:
import torch
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO

# ===========================
# 1. Load the YOLO Model
# ===========================
model_path = "C:/Ml_model_train/runs/detect/train/weights/best.pt"
model = YOLO(model_path)

# ===========================
# 2. Load Test Images
# ===========================
test_images_folder = "C:/Ml_model_train/test/images"
output_folder = "C:/Ml_model_train/test_results"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get list of test images
image_paths = [os.path.join(test_images_folder, img) for img in os.listdir(test_images_folder) if img.endswith(('.jpg', '.png', '.jpeg'))]

# ===========================
# 3. Run Inference and Visualize
# ===========================
for image_path in image_paths:
    # Read image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for visualization

    # Run YOLO inference
    results = model.predict(image_path)

    # Process results and draw bounding boxes
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding box coordinates (x1, y1, x2, y2)
        classes = result.boxes.cls.cpu().numpy().astype(int)  # Class indices
        confidences = result.boxes.conf.cpu().numpy()  # Confidence scores
        class_names = result.names  # Class labels

        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = map(int, box)
            label = f"{class_names[cls]}: {conf:.2f}"
            
            # Draw bounding box
            cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Put label text
            cv2.putText(img_rgb, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # ===========================
    # 4. Display and Save Results
    # ===========================
    plt.figure(figsize=(8, 6))
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.title(f"YOLO Prediction: {os.path.basename(image_path)}")
    plt.show()  # Display the image

    # Save the image with predictions
    save_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(save_path, cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))
    print(f"Saved: {save_path}")



image 1/1 C:\Ml_model_train\test\images\Tr-gl_0016_jpg.rf.99746694ea97fe0b73108832b462d48e.jpg: 640x640 1 Glioma, 83.0ms
Speed: 5.1ms preprocess, 83.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x600 with 1 Axes>

Saved: C:/Ml_model_train/test_results\Tr-gl_0016_jpg.rf.99746694ea97fe0b73108832b462d48e.jpg

image 1/1 C:\Ml_model_train\test\images\Tr-gl_0018_jpg.rf.7a670766b8083a1b516a49e241a636bc.jpg: 640x640 1 Glioma, 8.4ms
Speed: 2.4ms preprocess, 8.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x600 with 1 Axes>

Saved: C:/Ml_model_train/test_results\Tr-gl_0018_jpg.rf.7a670766b8083a1b516a49e241a636bc.jpg

image 1/1 C:\Ml_model_train\test\images\Tr-gl_0028_jpg.rf.8098b9737048aac0dccb1b7c45046894.jpg: 640x640 1 Glioma, 8.2ms
Speed: 2.4ms preprocess, 8.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


...

Saved: C:/Ml_model_train/test_results\Tr-pi_0636_jpg.rf.609ee034c63ba7e5b9f443de105b1b58.jpg

image 1/1 C:\Ml_model_train\test\images\Tr-pi_0646_jpg.rf.d3e36be566da199204d83608fc892ec0.jpg: 640x640 1 Pituitary, 16.7ms
Speed: 3.0ms preprocess, 16.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x600 with 1 Axes>

Saved: C:/Ml_model_train/test_results\Tr-pi_0646_jpg.rf.d3e36be566da199204d83608fc892ec0.jpg

image 1/1 C:\Ml_model_train\test\images\Tr-pi_0657_jpg.rf.1be85a960e376fd98fa1311070225b19.jpg: 640x640 1 Pituitary, 16.7ms
Speed: 3.6ms preprocess, 16.7ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 800x600 with 1 Axes>

Saved: C:/Ml_model_train/test_results\Tr-pi_0657_jpg.rf.1be85a960e376fd98fa1311070225b19.jpg
